In [1]:
import asyncio
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.kernel import Kernel

In [2]:
# Define the agent name and instructions
AGENT_NAME = "Host"
AGENT_INSTRUCTIONS = "Answer questions about the travel."


# Define a sample plugin for the sample
class TravelPlugin:
    """A sample Travel Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of countries.")
    def get_countries(self) -> Annotated[str, "Returns the countries that you can book."]:
        return """
            United States
            Australia
            France
        """

    @kernel_function(description="Provides the typical travel budget for a country")
    def get_travel_budget(self, 
        country: Annotated[str, "The name of the country"]) -> Annotated[str, "Returns the budget for the country."]:
        if country == "United States":
            return """
                Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
                Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
            """
        elif country == "Australia":
            return """
                Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
                Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
            """
        elif country == "France":
            return """
                Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
                Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.            
            """
        else:
            return "Country not found"
    
    @kernel_function(description="Provides the weather for a country")
    def get_weather(self, 
        country: Annotated[str, "The name of the country"]) -> Annotated[str, "Returns the weather for the country."]:
        if country == "United States":
            return """
                The United States has a diverse climate. In general:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
                - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
            """
        elif country == "Australia":
            return """
                Australia experiences varied climates:
                - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
                - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
            """
        elif country == "France":
            return """
                France has several climate zones:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
                - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
            """
        else:
            return "Country not found"
    

In [3]:
# A helper method to invoke the agent with the user input
async def invoke_agent(agent: ChatCompletionAgent, input: str, chat: ChatHistory) -> None:
    """Invoke the agent with the user input."""
    chat.add_user_message(input)

    print(f"# {AuthorRole.USER}: '{input}'")
    
    async for content in agent.invoke(chat):
        print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
        chat.add_message(content)
        await asyncio.sleep(5)


In [4]:
# Create the instance of the Kernel
kernel = Kernel()

service_id = "agent"
kernel.add_service(AzureChatCompletion(service_id=service_id))

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

kernel.add_plugin(TravelPlugin(), plugin_name="menu")

# Create the agent
agent = ChatCompletionAgent(
    service_id="agent", kernel=kernel, name=AGENT_NAME, instructions=AGENT_INSTRUCTIONS, execution_settings=settings
)

# Define the chat history
history = ChatHistory()

is_complete: bool = False
while not is_complete:
    user_input = input("User:> ")
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        
        break

    await invoke_agent(agent, user_input, history)


# AuthorRole.USER: 'what countries can i go to?'
# AuthorRole.ASSISTANT - Host: 'You can travel to the following countries:
- United States
- Australia
- France'
# AuthorRole.USER: 'I dont have much money, how much do i need to travel in France for 4 days?'
# AuthorRole.ASSISTANT - Host: 'For a 4-day trip to France, here's an estimate of how much money you'll need based on your travel style:

- **Budget Travelers**: Around $100 per day.
  - Total for 4 days: **$400**
  
- **Mid-Range Travelers**: Approximately $286 per day.
  - Total for 4 days: **$1144**
  
- **Luxury Travelers**: About $908 per day.
  - Total for 4 days: **$3632**

If you don't have much money, you could plan for a budget travel style, which would be around $400 for 4 days.'
# AuthorRole.USER: 'i only have 300 USD. where can i go for 4 days'
# AuthorRole.ASSISTANT - Host: 'Based on your budget of $300 for 4 days, here's what you can consider:

### United States:
- **Budget Travelers**: Around $121 per day.
  - Total 